In [3]:
import numpy
import scipy.special
import matplotlib.pyplot
%matplotlib inline
class neuralNetwork:
    def __init__(self, imputnodes, hiddennodes, outputnodes, learningrate):
        self.inodes = imputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        self.lr = learningrate
        
        self.wih = numpy.random.normal(0.0, pow(self.hnodes, -0.5),(self.hnodes, self.inodes))

        self.who = numpy.random.normal(0.0, pow(self.onodes, -0.5),(self.onodes, self.hnodes))

        
        self.activation_function = lambda x: scipy.special.expit(x)
        pass
    def train (self, inputs_list, targets_list) :
        # преобразовать список входных значений в двухмерный массив
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # рассчитать входящие сигналы для скрытого слоя
        hidden_inputs = numpy.dot(self.wih, inputs)
        # рассчитать исходящие сигналы для скрытого слоя
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # рассчитать входящие сигналы для выходного слоя
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # рассчитать исходящие сигналы для выходного слоя
        final_outputs = self.activation_function(final_inputs)
        
        output_errors = targets - final_outputs
        hidden_errors = numpy.dot(self.who.T, output_errors)
        
        #Обновляем весовые коэффициенты связей между скрытыми и выходными слоями
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        #Обновляем весовые коэффициенты связей между входным и скрытым слоями
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        pass

    def query(self, inputs_list):
        #входные значения
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        
        #входящие сигналы скрытого слоя
        hidden_inputs = numpy.dot(self.wih, inputs)
        #выходящие сигналы скрытого слоя
        hidden_outputs = self.activation_function(hidden_inputs)
        
        #входящие сигналы для выходного слоя
        final_inputs = numpy.dot(self.who, hidden_outputs)
        #выходящие сигналы выходного слоя
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs
        

In [4]:
training_data_file = open("emnist-letters-train.csv", "r")
training_data_list = training_data_file.readlines()
training_data_file.close()
# кол-во узлов на каждом слое
input_nodes = 784
hidden_nodes = 500
output_nodes = 26
# коэффициент обучения равен 0,1
learning_rate =0.05
# создать экземпляр нейронной сети
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes,learning_rate)
#задаем кол-во кругов тренировки на одних данных
epoсh = 1
while epoсh <= 1:
    epoсh = epoсh + 1;
    #ТРЕНИРОВКА
    # перебрать все записи в тренировочном наборе данных
    for record in training_data_list:
        all_values = record.split(',')
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # создать целевые выходные значения (все равны 0,01, за исключением
        # желаемого маркерного значения, равного 0,99)
        targets = numpy.zeros(output_nodes) + 0.01
        #all_values[0] - целевое маркерное значение для данной записи
        targets[int(all_values[0])-1] = 0.99
        n.train(inputs, targets)
        pass

In [176]:
# ТЕСТ
# журнал оценок работы сети, первоначально пустой
scorecard = []

test_data_file = open("emnist-letters-test.csv", "r")
test_data_list = test_data_file.readlines()
test_data_file.close()



for record in test_data_list:
    # получить список значений из записи, используя символы
    # запятой (*,1) в качестве разделителей
    all_values = record.split(',')
    # правильный ответ - первое значение
    correct_label = int(all_values[0])-1
    # масштабировать и сместить входные значения
    test_inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    # опрос сети
    outputs = n.query(test_inputs)
    # индекс наибольшего значения является маркерным значением
    label = numpy.argmax(outputs)
    # присоединить оценку ответа сети к концу списка
    if (label == correct_label) :
        # в случае правильного ответа сети присоединить
        # к списку значение 1
        scorecard.append(1)
    else:
        # в случае неправильного ответа сети присоединить
        # к списку значение 0
        scorecard.append(0)
        pass
    pass

# рассчитать показатель эффективности в виде
# доли правильных ответов
scorecard_array = numpy.asarray(scorecard)
print ("эффективность = ", (scorecard_array.sum() /  scorecard_array.size)*100)

эффективность =  83.39189189189189


In [5]:
import numpy as np
from PIL import ImageTk, Image, ImageDraw
import PIL
import tkinter as tk
from tkinter import *

letters = ['A', 'B', 'C', 'D','E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V','W', 'X', 'Y', 'Z']

#Задаем ширину и высоту окна для ввода 
width = 280
height = 280
center = height//2
white = (255, 255, 255)

#рисование
def paint(event):
    x1, y1 = (event.x - 10), (event.y - 10)
    x2, y2 = (event.x + 10), (event.y + 10)
    cv.create_oval(x1, y1, x2, y2, fill="black",width=10)
    draw.line([x1, y1, x2, y2],fill="black",width=10)
    
#преобразовываем изображение в массив и подаем сети
def pridiction():
    filename = "image.jpg"
    image1.save(filename)
    img = image1
    #Перевод в серый цвет 
    img = PIL.Image.open(filename).convert("L") 
    #изменняем размер изображения 
    img = img.resize((28,28))
    #переводим изображение в массив
    im2arr = np.array(img)
    im2arr = 255 - im2arr  
    # преобразовывем массив
    im2arr = im2arr.T
    im2arr = im2arr.reshape(784,)
    img = im2arr.reshape((28,28))
    #преобразование входных данных для опроса сетью
    inputs = ((im2arr / 255) * 0.98 + 0.01) 
    outputs = n.query(inputs)
    label = numpy.argmax(outputs)
    #вывод резулльатов на экран
    txt.insert(tk.INSERT,"{}".format(letters[label]))
    accuracy.insert(tk.INSERT,('%d%%' %(round(outputs[label][0]*100))))
    
# очищаем поле ввода и поля вывода
def clear():
    cv.delete('all')
    draw.rectangle((0, 0, 280, 280), fill=(255, 255, 255, 0))
    txt.delete('1.0', END)
    accuracy.delete('1.0', END)
    
root = Tk()

root.geometry('300x410')


#создаем поле для рисования(холст)
cv = Canvas(root, width=width, height=height, bg='white')
# PIL создает пустое изображение и русет на нем
image1 = PIL.Image.new("RGB", (width, height), white)
draw = ImageDraw.Draw(image1)
#само рисование на холсте
cv.bind("<B1-Motion>", paint)
#Параметры текста вывода
txt=tk.Text(root,bd=2,exportselection=2,bg='WHITE',font='Helvetica',
            padx=2,pady=2,height=1,width=2)
accuracy = tk.Text(root,bd=3,exportselection=1,bg='WHITE',font='Helvetica',
            padx=1,pady=2,height=1,width=4)

what_do = "Нарисуйте латинскую букву во всё поле:"
txt_accuracy = "Точность:"
pred = "Предсказание:"
label1 = Label(text=what_do, justify=LEFT, height = 1)
label2 = Label(text=txt_accuracy, justify=LEFT, height = 1)
label3 = Label(text=pred, justify=LEFT, height = 1)

#Привязка функций кнопкам
btnPred = Button(root, text = "Predict",width = 6,bd =3,font="Arial 11", command = lambda:pridiction())
btnClear = Button(root, text = "Clear",width = 6,bd =3,font="Arial 11", command = lambda:clear())
#Задаем координаты кнопок и полей
label1.place(x =150, y =13, anchor = CENTER)
cv.place(x= 10, y =30)
btnPred.place(x = 60, y = 365)
btnClear.place(x = 160, y = 365)
txt.place(x = 100, y = 320)
accuracy.place(x = 222, y = 320)
label2.place(x = 150, y = 324)
label3.place(x = 10, y = 323)
root.title('Латинские буквы')

root.mainloop()